To build chatbot that can interact with users in a conversational manner using pre-trained LLM and LangChain ConversationChain

!pip install transformers torch langchain sentencepiece

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms.base import LLM
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Step 1: Load Hugging Face Model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Use Hugging Face pipeline for text generation
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,
    do_sample=True,
    temperature=0.7
)

# Step 2: Wrap as LangChain LLM
class HuggingFaceLLM(LLM):
    """LangChain wrapper for Hugging Face transformers"""

    @property
    def _llm_type(self):
        return "huggingface"

    def _call(self, prompt: str, stop=None):
        output = generator(prompt, max_length=200, num_return_sequences=1)
        return output[0]["generated_text"]

# Instantiate the LLM
llm = HuggingFaceLLM()

# Step 3: Setup ConversationChain
memory = ConversationBufferMemory()
chatbot = ConversationChain(llm=llm, memory=memory)

# Step 4: Run Chat Loop
print("Chatbot is ready! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break
    response = chatbot.run(user_input)
    print("Chatbot:", response)